In [ ]:
import pandas as pd
from docx import Document
import openai
import json

In [147]:
df = pd.read_parquet("data/data.parquet")

In [148]:
sector= pd.read_excel('data/diccionario.xlsx',sheet_name='Sector',skiprows=2)
sector.columns = sector.iloc[0]
sector.drop(index=0, inplace=True)
sector.reset_index(drop=True,inplace=True)
sector['Código'] =sector['Código'].astype(int)

In [149]:
depa= pd.read_excel('data/diccionario.xlsx',sheet_name='Departamento',skiprows=3)
depa = depa[['Código','DEPARTAMENTO']].loc[:24,:]
depa['Código'] = depa['Código'].astype(int)

In [150]:
doc = Document('plantilla.docx')

In [151]:
def replace_text(doc, replacements):
    def replace_in_paragraph(paragraph, replacements):
        full_text = ''.join(run.text for run in paragraph.runs)
        
        for old_text, new_text in replacements.items():
            if old_text in full_text:
                full_text = full_text.replace(old_text, new_text)
        
        for run in paragraph.runs:
            run.clear()

        paragraph.add_run(full_text)

    for paragraph in doc.paragraphs:
        replace_in_paragraph(paragraph, replacements)

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    replace_in_paragraph(paragraph, replacements)

In [152]:
def convertir_fecha(valor):
    valor_str = str(valor)
    año = valor_str[:4]
    mes_num = valor_str[4:6]
    
    meses = {
        '01': 'Enero', '02': 'Febrero', '03': 'Marzo', '04': 'Abril',
        '05': 'Mayo', '06': 'Junio', '07': 'Julio', '08': 'Agosto',
        '09': 'Septiembre', '10': 'Octubre', '11': 'Noviembre', '12': 'Diciembre'
    }
    
    mes_nombre = meses.get(mes_num, 'Mes inválido')
    
    return f"{año} - {mes_nombre}"

In [153]:
def return_sector(id):
    try:
        return f"{sector[sector['Código']==id]['Descripción'].values[0]}"
    except:
        return '-'

In [154]:
def return_depa(id):
    try:
        return f"{depa[depa['Código']==id]['DEPARTAMENTO'].values[0]}"
    except:
        return '-'

In [155]:
def seleccionar_fila(busqueda):
    try:
        row = df[df['ID_EMPRESA']==busqueda]
        print('Sí se encontró la empresa, generando Planilla Electrónica...')
        return row
    except:
        return print('No se encontró la empresa')

In [ ]:
def openai_obs(key,prompt):
    openai.api_key = key

    system_message = '.'

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': prompt}
        ]
    )

    assistant_response = response.choices[0].message['content']

    output_data = {
        'análisis': assistant_response
    }

    with open('response.json', 'w', encoding='utf-8') as json_file:
        json.dump(output_data, json_file, ensure_ascii=False, indent=4)

    print("El análisis ha sido guardado en 'response.json'")
    with open('response.json', 'r', encoding='utf-8') as json_file:
        response = json.load(json_file)

    list_obs = []

    for key, value in response['análisis'].items():
        if isinstance(value, dict) and 'observación' in value:
            list_obs.append(f"• {value['observación']}")

    obs = '\n\n'.join(list_obs)

    return obs

El análisis ha sido guardado en 'response.json'


In [157]:
def process_text(row):
    Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7 = convertir_fecha(row['V_NUMPERIOD'].values[0]), row['ID_EMPRESA'].values[0],return_depa(row['Departamento_ct'].values[0]), return_sector(row['sector'].values[0]), row['anio_anti'].values[0],row['numtra'].values[0], row['costosal'].values[0], row['sst'].values[0]
    
    # Variables A,B
    A0, A1, A2, A3, A4, A5 = row['ntrab_rl_privgen'].values[0], row['ntrab_rl_peq'].values[0], row['ntrab_rl_micro'].values[0], row['ntrab_rl_agrar'].values[0], row['ntrab_rl_otros'].values[0], row['ntrab_rl_noprec'].values[0]
    suma_A = A0 + A1 + A2 + A3 + A4 + A5
    B0, B1, B2, B3, B4, B5 = [round(round(val / suma_A, 4) * 100,2) for val in (A0, A1, A2, A3, A4, A5)]

    # Variables C,D
    C0, C1, C2, C3, C4, C5, C6 = row['ntrab_tc_indet'].values[0], row['ntrab_tc_nattemp'].values[0], row['ntrab_tc_natacc'].values[0], row['ntrab_tc_obrserv'].values[0], row['ntrab_tc_tiempar'].values[0], row['ntrab_tc_otros'].values[0], row['ntrab_tc_noprec'].values[0]
    suma_C = C0 + C1 + C2 + C3 + C4 + C5 + C6
    D0, D1, D2, D3, D4, D5, D6 = [round(round(val / suma_C, 4) * 100,2) for val in (C0, C1, C2, C3, C4, C5, C6)]

    # Variables E,F
    E0,E1,E2,E3 = row['ntrab_rp_spp'].values[0], row['ntrab_rp_snp'].values[0], row['ntrab_rp_otros'].values[0], row['ntrab_rp_noprecrp'].values[0]
    suma_E = E0 + E1 + E2 + E3
    F0,F1,F2,F3 = [round(round(val / suma_E, 4) * 100,2) for val in (E0,E1,E2,E3)]

    # Variables G,H
    G0,G1,G2,G3 = row['ntrab_afil_essalud'].values[0], row['ntrab_afil_essalud_agrar'].values[0], row['ntrab_afil_sis'].values[0], row['ntrab_afil_noprec'].values[0]
    suma_G = G0 + G1 + G2 + G3
    H0, H1, H2, H3 = [round(round(val / suma_G, 4) * 100,2) for val in (G0,G1,G2,G3)]

    # Variables I,J
    I0,I1,I2 = row['ntrab_18mas'].values[0], row['ntrab_menos18'].values[0], row['ntrab_edad_noprec'].values[0]
    suma_I = I0 + I1 + I2
    J0, J1, J2 = [round(round(val / suma_I, 4) * 100,2) for val in (I0,I1,I2)]

    # Variables K,L
    K0,K1,K2 = row['ntrab_mujeres'].values[0], row['ntrab_hombres'].values[0], row['ntrab_sexo_noprec'].values[0]
    suma_K = K0 + K1 + K2
    L0, L1, L2 = [round(round(val / suma_K, 4) * 100,2)for val in (K0, K1, K2)]

    # Variables M,N
    M0, M1 = row['ntrab_sind'].values[0], row['ntrab_nosind'].values[0]
    suma_M = M0 + M1
    N0, N1 = [round(round(val / suma_M, 4) * 100,2) for val in (M0, M1)]

    # Variables O,P
    O0, O1, O2 = row['ntrab_consctr'].values[0], row['ntrab_sinsctr'].values[0], row['ntrab_sexo_noprec'].values[0]
    suma_O = O0 + O1 + O2
    P0, P1, P2 = [round(round(val / suma_O, 4) * 100,2) for val in (O0, O1, O2)]
    
    # Diccionario de reemplazos
    reemplazos = {
        'Z0': str(Z0),
        'Z1': str(Z1),
        'Z2': str(Z2),
        'Z3': str(Z3),
        'Z4': str(Z4),
        'Z5': str(Z5),
        'Z6': str(Z6),
        'Z7': str(Z7),
        'A0': str(A0),
        'A1': str(A1),
        'A2': str(A2),
        'A3': str(A3),
        'A4': str(A4),
        'A5': str(A5),
        'B0': str(B0),
        'B1': str(B1), 
        'B2': str(B2),
        'B3': str(B3),
        'B4': str(B4),
        'B5': str(B5),
        'C0': str(C0),
        'C1': str(C1),
        'C2': str(C2),
        'C3': str(C3),
        'C4': str(C4),
        'C5': str(C5),
        'C6': str(C6),
        'D0': str(D0),
        'D1': str(D1),
        'D2': str(D2),
        'D3': str(D3),
        'D4': str(D4),
        'D5': str(D5),
        'D6': str(D6),
        'E0': str(E0),
        'E1': str(E1),
        'E2': str(E2),
        'E3': str(E3),
        'F0': str(F0),
        'F1': str(F1),
        'F2': str(F2),
        'F3': str(F3),
        'G0': str(G0),
        'G1': str(G1),
        'G2': str(G2),
        'G3': str(G3),
        'H0': str(H0),
        'H1': str(H1),
        'H2': str(H2),
        'H3': str(H3),
        'I0': str(I0),
        'I1': str(I1),
        'I2': str(I2),
        'J0': str(J0),
        'J1': str(J1),
        'J2': str(J2),
        'K0': str(K0),
        'K1': str(K1),
        'K2': str(K2),
        'L0': str(L0),
        'L1': str(L1),
        'L2': str(L2),
        'M0': str(M0),
        'M1': str(M1),
        'N0': str(N0),
        'N1': str(N1),
        'O0': str(O0),
        'O1': str(O1),
        'O2': str(O2),
        'P0': str(P0),
        'P1': str(P1),
        'P2': str(P2)
    }
    
    
    replace_text(doc, reemplazos)
    
    doc.save(f"Planilla_Electrónica_{Z1}.docx")
    print('Documento Generado')
    
    temp_dicc = {'regimen laboral': suma_A,
                'tipo de contrato':suma_C,
                'régimen pensionario': suma_E,
                'afiliación a servicios de salud':suma_G,
                'edad':suma_I,
                'genero':suma_K,
                'sindicalización':suma_M,
                'SCTR':suma_O}
    

    temp = [col for col,value in temp_dicc.items() if value!=Z5]
    temp2 = [str(value) for col,value in temp_dicc.items() if value!=Z5]
    temp_msg = ', '.join(temp)
    temp_val = ', '.join(temp2)


    prompt = f"""
    Analiza los datos de la empresa '{Z1}' y proporciona observaciones o patrones relevantes:

    - Declaración: {Z0}, Sector: {Z3}, Departamento: {Z2}
    - Contratos: Indefinidos {C0}, Temporales {C1}, Accidentales {C2}, Obra {C3}, Parcial {C4}, Otros {C5}, No especificado {C6}
    - Régimen: Privado {A0}, Micro {A2}, Pequeña {A1}, Agrario {A3}, Otros {A4}, No especificado {A5}
    - Pensión: SPP {E0}, SNP {E1}, Otros {E2}, No especificado {E3}
    - Salud: EsSalud {G0}, Agrario {G1}, SIS {G2}, No especificado {G3}
    - Edad: 18+ {I0}, <18 {I1}, No especificado {I2}
    - Sexo: Hombres {K1}, Mujeres {K0}, No especificado {K2}
    - Sindicalización: Sí {M0}, No {M1}
    - SCTR: Con {O0}, Sin {O1}, No especificado {O2}
    - Total Trabajadores: {Z5}, Costo Salarial: {Z6}, Antigüedad: {Z4}, SST: {'Sí' if Z7 == 1 else 'No'}

    Objetivo:
        1. Identificar patrones en la estructura de contratos, régimen laboral, salud, edad y sindicalización.
        2. Observar los costos y la antigüedad.
        3. Detectar posibles irregularidades o sugerencias de mejora.
    """
    
    with open('keys.json', 'r') as json_file:
        keys = json.load(json_file)
        key = keys.get('key') 


    if temp:
        msg = {
            'Y0':f"• Se ha detectado una discrepancia en el número de trabajadores registrados: {Z5}, respecto al total esperado para {temp_msg} ({temp_val}). Se recomienda revisar y ajustar este valor a la mayor brevedad.\n",
            'Y1':openai_obs(key,prompt)}

    else:
        msg = {'Y0':f"• No se ha detectado una discrepancia en el número de trabajadores registrados\n\n",
                'Y1':openai_obs(key,prompt)}

    replace_text(doc,msg)

In [158]:
row = seleccionar_fila('6UV9S7A9Q3E')
process_text(row)

Sí se encontró la empresa, generando Planilla Electrónica...
Documento Generado


In [159]:
from docx.shared import Pt 

def apply_format_to_cell(cell, bold=False, font_size=None):
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            if bold:
                run.bold = True
            if font_size:
                run.font.size = Pt(font_size)

def apply_format_to_text(doc, text_to_format, bold=False, font_size=None):
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if text_to_format in run.text:
                if bold:
                    run.bold = True
                if font_size:
                    run.font.size = Pt(font_size)

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        if text_to_format in run.text:
                            if bold:
                                run.bold = True
                            if font_size:
                                run.font.size = Pt(font_size)

formatos = [
    ("Planilla Electrónica", {'bold': False, 'font_size': 18}),
    ("Empresa:", {'bold': True, 'font_size': 12}),
    ("Departamento", {'bold': True, 'font_size': 12}),
    ("Sector", {'bold': True, 'font_size': 12}),
    ("Años en el mercado", {'bold': True, 'font_size': 12}),
    ("Costo Salarial", {'bold': True, 'font_size': 12}),
    ("SST", {'bold': True, 'font_size': 12}),
    ("Nº de trabajadores", {'bold': True, 'font_size': 12}),
    ("Régimen Laboral", {'bold': True, 'font_size': 11}),
    ("Régimen Pensionario", {'bold': True, 'font_size': 11}),
    ("Afiliación Sindical", {'bold': True, 'font_size': 11}),
    ("Rango de Edades", {'bold': True, 'font_size': 11}),
    ("Tipo de Contrato", {'bold': True, 'font_size': 11}),
    ("Régimen de Salud", {'bold': True, 'font_size': 11}),
    ("Género", {'bold': True, 'font_size': 11}),
    ("Afiliación al SCTR", {'bold': True, 'font_size': 11}),
    ("N° Trabj.", {'bold': True, 'font_size': 11}),
    ("% Trabj.", {'bold': True, 'font_size': 11}),
    ("Observaciones:", {'bold': True, 'font_size': 16})
]

for texto, propiedades in formatos:
    apply_format_to_text(doc, texto, **propiedades)

In [160]:
doc.save(f"planilla_Electrónica_{row['ID_EMPRESA'].values[0]}.docx")

In [161]:
from docx2pdf import convert

nombre_docx = f"planilla_Electrónica_{row['ID_EMPRESA'].values[0]}.docx"
convert(nombre_docx)

  0%|          | 0/1 [00:00<?, ?it/s]